In [17]:
# test maridb에 test 해보기  #
# 실시간 동기화까지는 어렵고 , 동기화 느낌이 나게 일정간격동안 체크하고 체크 조건에 안맞으면 강제 업데이트 해버림 

import jaydebeapi
import pymysql 
import pandas as pd 
from pandas import json_normalize
import json

In [3]:
# 티베로 db 기준으로 데이터 확인 
tibero_conn = jaydebeapi.connect(
    "com.tmax.tibero.jdbc.TbDriver",
    "jdbc:tibero:thin:@172.7.0.23:8629:tibero",
    ["labelon", "euclid!@)$labelon"],
    "C:/Users/yoonsub/tibero6-jdbc.jar",
)
tibero_cur = conn.cursor()

In [11]:
# 마리아db 연결  저장할 db 
maria_conn = pymysql.connect(host = '127.0.0.1' , user = 'root' , password = '1234' , db = 'labelon_dummy', charset='utf8' )
maria_cur = maria_conn.cursor()


In [3]:
# #실제 db 입력 쿼리대신 db에서 json 파일 로드 후 사용 
# json_file = "C:/Users/yoonsub/MANAGE_PHYSICAL_TABLE.json" # table
# with open(json_file, 'r', encoding='utf8') as f:
#     json_data_table = json.load(f)

# json_file = "C:/Users/yoonsub/MANAGE_PHYSICAL_COLUMN.json" # column
# with open(json_file, 'r', encoding='utf8') as e:
#     json_data_colum = json.load(e)

# df1 = json_normalize(json_data_table)
# df2 = json_normalize(json_data_colum)


# print(len(df1.iloc[:]))


10186


In [7]:
# 티베로 db select 후 비교 

select_table_sql = 'select * from manage_physical_table'
select_column_sql = 'select * from manage_physical_column'
df1 = pd.read_sql_query(select_table_sql , tibero_conn)
df2  = pd.read_sql_query(select_column_sql , tibero_conn)



C:\ProgramData\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [9]:
print(len(df1.iloc[:]))

10251


In [12]:
# db 정보 비교 

db_len = len(df1.iloc[:])


count_mairadb_sql = "select id from manage_physical_table"
maria_cur.execute(count_mairadb_sql)
count = len(maria_cur.fetchall())

# TRUNCATE 
truncate_sql_table = "TRUNCATE TABLE manage_physical_table"
truncate_sql_column = "TRUNCATE TABLE manage_physical_column"

if db_len != count : 
        print("오리지널 db와 행불일치로 데이터를 지우고 다시 insert 합니다. ")
        maria_cur.execute(truncate_sql_table)
        maria_cur.execute(truncate_sql_column)
        
else: 
    print("동일")
                      



오리지널 db와 행불일치로 데이터를 지우고 다시 insert 합니다. 


In [13]:
# df1 table 정보 몇가지를 mysql로 옮긴다. 
insert_data = df1.iloc[1, 0:5]
insert_data


ID                                    1805
DATA_BASIC_ID                          219
LOGICAL_TABLE_KOREAN     서울일자리센터 우수중소기업 정보
LOGICAL_TABLE_ENGLISH     TN_SMEP_REQ_OPEN
PHYSICAL_TABLE_NAME          NLDATA_000219
Name: 1, dtype: object

In [14]:
# table 정보 입력 , 실행시 테이블 비우기 후 입력되게 설계 필요 
for i in range(len(df1.iloc[:])) : 
    ID = df1.iloc[i , 0]
    DATA_BASIC_ID = df1.iloc[i , 1]
    LOGICAL_TABLE_KOREAN = df1.iloc[i , 2]
    LOGICAL_TABLE_ENGLISH = df1.iloc[i , 3]
    PHYSICAL_TABLE_NAME = df1.iloc[i , 4]

    mysql_insert_sql = f"INSERT INTO MANAGE_PHYSICAL_TABLE (ID, DATA_BASIC_ID, LOGICAL_TABLE_KOREAN, LOGICAL_TABLE_ENGLISH, PHYSICAL_TABLE_NAME) VALUE ('{ID}', '{DATA_BASIC_ID}', '{LOGICAL_TABLE_KOREAN}','{LOGICAL_TABLE_ENGLISH}','{PHYSICAL_TABLE_NAME}')" 
    maria_cur.execute(mysql_insert_sql)
    maria_conn.commit()       
        

In [15]:
# df2 table 정보 몇가지를 mysql로 옮긴다. 
insert_data02 = df2.iloc[1, 0:5]
insert_data02


ID                            28381
DATA_PHYSICAL_ID               1495
LOGICAL_COLUMN_KOREAN      노드 유형 코드
LOGICAL_COLUMN_ENGLISH    NODE_CODE
PHYSICAL_COLUMN_NAME        COL_004
Name: 1, dtype: object

In [16]:
# table 정보 입력 , 실행시 테이블 비우기 후 입력되게 설계 필요 


for i in range(len(df2.iloc[:])) : 
    try : 
        ID = df2.iloc[i , 0]
        DATA_PHYSICAL_ID = df2.iloc[i , 1]
        LOGICAL_COLUMN_KOREAN = df2.iloc[i , 2]
        LOGICAL_COLUMN_ENGLISH = df2.iloc[i , 3]
        LOGICAL_COLUMN_ENGLISH = str(LOGICAL_COLUMN_ENGLISH)
        PHYSICAL_COLUMN_NAME  = df2.iloc[i , 4]

        mysql_insert_sql = f"INSERT INTO MANAGE_PHYSICAL_COLUMN (ID, DATA_PHYSICAL_ID, LOGICAL_COLUMN_KOREAN, LOGICAL_COLUMN_ENGLISH, PHYSICAL_COLUMN_NAME ) VALUE ('{ID}', '{DATA_PHYSICAL_ID }', '{LOGICAL_COLUMN_KOREAN}','{LOGICAL_COLUMN_ENGLISH}','{PHYSICAL_COLUMN_NAME}')" 
        maria_cur.execute(mysql_insert_sql)
        maria_conn.commit()
    except : 
        continue 
        


